##### 데이터 가져오기(사람인)

In [35]:
import pandas as pd

df = pd.read_csv("./saramin/saramin_deeplearning.csv")
df.head()

,기업이름,고용형태,연봉,기업주소,키워드,기업형태,사원수,매출액
0,미디어젠(주),신입 · 경력 · 정규직,면접 후 결정,"경기 성남시 분당구 성남대로331번길 8, 11층 1103호",딥러닝,"코스닥, 중소기업, 외부감사법인, 연구소..업종응용 소프트웨어 개발 및 공급업",189,-
1,(주)에스피파트너스,신입 · 경력 · 정규직,면접 후 결정,"경북 경산시 펜타힐즈2로 59, 802호",딥러닝,"중소기업, 주식회사업종시스템 소프트웨어 개발 및 공급업",22,"34억 5,777만"
2,디아이티(주),신입 · 경력 · 정규직,면접 후 결정,경기 화성시 정남면 가장로 355-26,딥러닝,"코스닥, 중소기업, 외부감사법인, 병역특..업종디스플레이 제조용 기계 제조업",194,-
3,(주)코난테크놀로지,신입 · 경력 · 정규직,회사내규에 따름,"서울 서초구 강남대로 327, 6층, 7층, 9층 (서초동,대륭서초타워)",딥러닝,"코스닥, 중소기업, 외부감사법인, 주식회..업종응용 소프트웨어 개발 및 공급업",270,-
4,(주)뷰온,경력무관 · 정규직,면접 후 결정,"서울 금천구 가산디지털1로 128, 806호, 807호, 808호",딥러닝,"중소기업, 병역특례 인증업체업종기타 광학기기 제조업",43,"77억 9,954만"


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829 entries, 0 to 828
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   기업이름    829 non-null    object
 1   고용형태    829 non-null    object
 2   연봉      829 non-null    object
 3   기업주소    829 non-null    object
 4   키워드     829 non-null    object
 5   기업형태    829 non-null    object
 6   사원수     829 non-null    object
 7   매출액     829 non-null    object
dtypes: object(8)
memory usage: 51.9+ KB


In [2]:
import mysql.connector

conn = mysql.connector.connect(
    host = "database-1.c3q2igwk6510.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "root",
    password = "20030320",
    database = "project_merge"
)
cursor = conn.cursor(buffered=True)

In [6]:
create_table_query = """
CREATE TABLE saramin (
    `기업이름` VARCHAR(50),
    `고용형태` VARCHAR(50),
    `연봉` VARCHAR(100),
    `기업주소` VARCHAR(150),
    `키워드` VARCHAR(20),
    `기업형태` VARCHAR(200),
    `사원수` VARCHAR(20),
    `매출액` VARCHAR(30)
);
"""
cursor.execute(create_table_query)

In [38]:
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO saramin (`기업이름`, `고용형태`, `연봉`, `기업주소`, `키워드`, `기업형태`, `사원수`, `매출액`)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
    """
    cursor.execute(insert_query, (
        row['기업이름'], row['고용형태'], row['연봉'], row['기업주소'],
        row['키워드'], row['기업형태'], row['사원수'], row['매출액']
    ))

In [ ]:
cursor.execute("ALTER TABLE saramin ADD COLUMN 인덱스 INT;")

In [39]:
cursor.execute("SET @index = 0;")
cursor.execute("UPDATE saramin SET `인덱스` = @index:=@index+1;")

In [40]:
cursor.execute("select * from saramin order by 인덱스 desc limit 3;")

result = cursor.fetchall()
for row in result:
    print(row)

('코나모빌리티(주)', '4 ~ 13년 · 정규직', '회사내규에 따름', '서울 영등포구 은행로 3, 901호', '딥러닝', '중소기업, 연구소, 주식회사업종응용 소프트웨어 개발 및 공급업', '34', '-', 1274)
('(주)지엔아이티', '3 ~ 10년 · 정규직', '회사내규에 따름', '경기 김포시 고촌읍 장차로5번길 20, 4층', '딥러닝', '중소기업, 주식회사업종시스템 소프트웨어 개발 및 공급업', '25', '-', 1273)
('주식회사 마이너', '신입 · 경력 · 정규직', '회사내규에 따름', '서울 관악구 신림로 134', '딥러닝', '-', '-', '-', 1272)


In [41]:
conn.commit()

##### 데이터 가져오기 잡플래닛

In [2]:
import pandas as pd

df = pd.read_csv("job_planet.csv")
df.head()

,기업이름,분야,평점,연봉,2차산업군
0,(주)티몬,IT/웹/통신,2.8,4175,IT
1,(주)위메프,IT/웹/통신,2.6,3989,IT
2,(주)엘지유플러스,IT/웹/통신,3.2,4909,IT
3,삼성에스디에스(주),IT/웹/통신,3.5,5939,IT
4,(주)엘지씨엔에스,IT/웹/통신,3.2,5322,IT


In [3]:
#Company.csv minus the value of nan, minus the name of the company
df['기업이름'] = df['기업이름'].str.replace('(주)', '')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12626 entries, 0 to 12625
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   기업이름    12626 non-null  object 
 1   분야      12626 non-null  object 
 2   평점      12626 non-null  float64
 3   연봉      12626 non-null  int64  
 4   2차산업군   12626 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 493.3+ KB


In [8]:
create_table_query = """
CREATE TABLE job_planet (
    `기업이름` VARCHAR(50),
    `분야` VARCHAR(30),
    `평점` float,
    `연봉` int,
    `2차산업군` VARCHAR(30)
   
);
"""
cursor.execute(create_table_query)

In [11]:
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO job_planet (`기업이름`, `분야`, `평점`, `연봉`, `2차산업군`)
    VALUES (%s, %s, %s, %s, %s);
    """
    cursor.execute(insert_query, (
        row['기업이름'], row['분야'], float(row['평점']), int(row['연봉']), row['2차산업군']))

In [12]:
cursor.execute("select * from job_planet limit 3;")

result = cursor.fetchall()
for row in result:
    print(row)

('티몬 ', 'IT/웹/통신', 2.8, 4175, 'IT')
('위메프 ', 'IT/웹/통신', 2.6, 3989, 'IT')
('엘지유플러스 ', 'IT/웹/통신', 3.2, 4909, 'IT')


In [13]:
conn.commit()

In [14]:
cursor.execute("select * from job_planet limit 3;")

result = cursor.fetchall()
for row in result:
    print(row)

('티몬 ', 'IT/웹/통신', 2.8, 4175, 'IT')
('위메프 ', 'IT/웹/통신', 2.6, 3989, 'IT')
('엘지유플러스 ', 'IT/웹/통신', 3.2, 4909, 'IT')


##### sql에서 csv로 저장

In [44]:
query = "SELECT * FROM job_planet"
df = pd.read_sql(query, conn)

/tmp/ipykernel_8410/909939827.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [45]:
df.to_csv('job_planet.csv', index=False)

##### 기업 성장 지표 테이블

In [1]:
import pandas as pd

df = pd.read_csv("./rev_merge.csv")
df.head()

,기업이름,기업규모,매출액,평균연봉,자본총계1,자본총계2,자본총계3,매출액1,매출액2,매출액3,순이익1,순이익2,순이익3,기준년도,키워드
0,에이알247,중소기업,0,4108,0,0,0,0,0,0,0,0,0,0,자율주행
1,쓰리핸즈,소기업,0,0,0,0,0,0,0,0,0,0,0,0,자율주행
2,매버릭컨설팅,중소기업,0,0,0,0,0,0,0,0,0,0,0,0,자율주행
3,현대자동차,대기업,4915569300,7371,0,0,0,0,0,0,0,0,0,0,자율주행
4,코가로보틱스,중소기업,37179,4578,2398977,101575,77147,37179,156842,74946,-835304,60623,37414,2022,자율주행


In [4]:
import mysql.connector

conn = mysql.connector.connect(
    host = "database-1.c3q2igwk6510.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "root",
    password = "20030320",
    database = "project_merge"
)
cursor = conn.cursor(buffered=True)

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323 entries, 0 to 322
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   기업이름    323 non-null    object
 1   기업규모    316 non-null    object
 2   매출액     323 non-null    int64 
 3   평균연봉    323 non-null    int64 
 4   자본총계1   323 non-null    int64 
 5   자본총계2   323 non-null    int64 
 6   자본총계3   323 non-null    int64 
 7   매출액1    323 non-null    int64 
 8   매출액2    323 non-null    int64 
 9   매출액3    323 non-null    int64 
 10  순이익1    323 non-null    int64 
 11  순이익2    323 non-null    int64 
 12  순이익3    323 non-null    int64 
 13  기준년도    323 non-null    int64 
 14  키워드     323 non-null    object
dtypes: int64(12), object(3)
memory usage: 38.0+ KB


In [5]:
create_table_query = """
CREATE TABLE cor_growth(
    기업이름 VARCHAR(255),
    기업규모 VARCHAR(255),
    매출액 INT,
    평균연봉 INT,
    자본총계1 INT,
    자본총계2 INT,
    자본총계3 INT,
    매출액1 INT,
    매출액2 INT,
    매출액3 INT,
    순이익1 INT,
    순이익2 INT,
    순이익3 INT,
    기준년도 INT,
    키워드 VARCHAR(255)
);
"""
cursor.execute(create_table_query)

In [6]:
for index, row in df.iterrows():
    insert_query = f"""
    INSERT INTO cor_growth
    VALUES (
        '{row["기업이름"]}', '{row["기업규모"]}', {row["매출액"]}, {row["평균연봉"]},
        {row["자본총계1"]}, {row["자본총계2"]}, {row["자본총계3"]},
        {row["매출액1"]}, {row["매출액2"]}, {row["매출액3"]},
        {row["순이익1"]}, {row["순이익2"]}, {row["순이익3"]},
        {row["기준년도"]}, '{row["키워드"]}'
    );
    """
    cursor.execute(insert_query)

In [7]:
cursor.execute("ALTER TABLE cor_growth ADD COLUMN 인덱스 INT;")

In [8]:
cursor.execute("SET @index = 0;")
cursor.execute("UPDATE cor_growth SET `인덱스` = @index:=@index+1;")

In [10]:
conn.commit()